## Custom GNN Transformer architecture

In [1]:
import torch
import torch.nn as nn
import time
from torch import optim
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GATConv, GATv2Conv
import torchvision
from einops import rearrange
import wandb

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(x, **kwargs) + x

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(self, x, **kwargs):
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim)
        )

    def forward(self, x, **kwargs):
        return self.net(x)
    
class QuickFix(nn.Module):
    def __init__(self, dim, heads, fn):
        super().__init__()
        self.dim = dim
        self.heads = heads
        self.linear = nn.Linear(dim * heads, dim)
        self.fn = fn
        
    def forward(self, x, **kwargs):
        return self.linear(self.fn(x, **kwargs))

In [3]:
# Out implementation of GAT Transformer

class GATTransformer(torch.nn.Module):
    def __init__(self, dim, depth, heads, mlp_dim):
        super().__init__()
        self.layers = nn.ModuleList()
        self.depth = depth
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                Residual(PreNorm(dim, QuickFix(dim, heads, GATv2Conv(in_channels=dim, out_channels=dim, heads=heads, add_self_loops=False)))),
                Residual(PreNorm(dim, FeedForward(dim, mlp_dim)))
            ]))


    def forward(self, data):
        x, edge_index = data

        for attn, ff in self.layers:
            x = attn(x=x, edge_index=edge_index)
            x = ff(x)

        return x


In [4]:
class GraphViT(nn.Module):
    def __init__(self, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels=3):
        super().__init__()
        assert image_size % patch_size == 0, 'image dimensions must be divisible by the patch size'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = channels * patch_size ** 2
        
        self.dim = dim

        self.patch_size = patch_size
        self.num_patches = num_patches
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.transformer = GATTransformer(dim, depth, heads, mlp_dim)

        self.to_cls_token = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim, num_classes)
        )

    def forward(self, img, mask=None):
        p = self.patch_size

        x = rearrange(img, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = p, p2 = p)
        x = x.to(device)
        x = self.patch_to_embedding(x)
        
        cls_tokens = self.cls_token.expand(img.shape[0], -1, -1)
        cls_tokens = cls_tokens.to(device)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding
        
    
        x_shape = x.shape
        edge_index = torch.ones((2, x.shape[1]**2*x.shape[0]), dtype=torch.long)
        for i in range(x.shape[0]*x.shape[1]):
            for j in range(x.shape[1]):
                edge_index[0,i*x.shape[1] + j] = i
                edge_index[1,i*x.shape[1] + j] = x.shape[1]*(i // x.shape[1]) + j
        x = x.view((x.shape[0]*x.shape[1], x.shape[2]))
        edge_index = edge_index.to(device)

        
        x = self.transformer([x, edge_index])
        x = x.view(x_shape)
        
        x = self.to_cls_token(x[:,0])
        x = x.to(device)
        
        out = self.mlp_head(x)
        
        return out

In [5]:
torch.manual_seed(42)
wandb.init(entity = 'njmarko', project = 'gat')

DOWNLOAD_PATH = '/data/mnist'
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_TEST = 1000

transform_mnist = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize((0.1307,), (0.3081,))])

#train_set = torchvision.datasets.MNIST(DOWNLOAD_PATH, train=True, download=True,
#                                       transform=transform_mnist)
train_set = torchvision.datasets.MNIST(DOWNLOAD_PATH, train=True, download=True,
                                       transform=transform_mnist)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)

test_set = torchvision.datasets.MNIST(DOWNLOAD_PATH, train=False, download=True,
                                      transform=transform_mnist)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE_TEST, shuffle=True)
# test_loader = torch_geometric.loader.DataLoader(test_set, batch_size=BATCH_SIZE_TEST, shuffle=True)

wandb: Currently logged in as: njmarko. Use `wandb login --relogin` to force relogin


In [6]:

def train_epoch(model, optimizer, data_loader, loss_history):
    total_samples = len(data_loader.dataset)
    model.train()
    running_loss = 0.0

    for i, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = F.log_softmax(model(data), dim=1)
        target = target.to(device)
        output = output.to(device)
        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 100 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(total_samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_loader)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_history.append(loss.item())
            wandb.log({'train_loss': running_loss/2000})

In [7]:
def evaluate(model, data_loader, loss_history):
    model.eval()
    
    total_samples = len(data_loader.dataset)
    correct_samples = 0
    total_loss = 0

    with torch.no_grad():
        for data, target in data_loader:
            res = model(data)
            res = res.to(device)
            output = F.log_softmax(res, dim=1)
            target = target.to(device)
            output = output.to(device)
            loss = F.nll_loss(output, target, reduction='sum')
            _, pred = torch.max(output, dim=1)
            
            total_loss += loss.item()
            correct_samples += pred.eq(target).sum()

    avg_loss = total_loss / total_samples
    loss_history.append(avg_loss)
    print('\nAverage test loss: ' + '{:.4f}'.format(avg_loss) +
          '  Accuracy:' + '{:5}'.format(correct_samples) + '/' +
          '{:5}'.format(total_samples) + ' (' +
          '{:4.2f}'.format(100.0 * correct_samples / total_samples) + '%)\n')

In [8]:
N_EPOCHS = 25

start_time = time.time()
model = GraphViT(image_size=28, patch_size=7, num_classes=10, channels=1,
            dim=64, depth=6, heads=8, mlp_dim=128)
optimizer = optim.Adam(model.parameters(), lr=0.003)
model = model.to(device)

train_loss_history, test_loss_history = [], []
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_epoch(model, optimizer, train_loader, train_loss_history)
    evaluate(model, test_loader, test_loss_history)

print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1
[    0/60000 (  0%)]  Loss: 2.3717
[10000/60000 ( 17%)]  Loss: 0.3379
[20000/60000 ( 33%)]  Loss: 0.3228
[30000/60000 ( 50%)]  Loss: 0.1994
[40000/60000 ( 67%)]  Loss: 0.1917
[50000/60000 ( 83%)]  Loss: 0.1004

Average test loss: 0.1530  Accuracy: 9517/10000 (95.17%)

Epoch: 2
[    0/60000 (  0%)]  Loss: 0.1826
[10000/60000 ( 17%)]  Loss: 0.0944
[20000/60000 ( 33%)]  Loss: 0.0435
[30000/60000 ( 50%)]  Loss: 0.2422
[40000/60000 ( 67%)]  Loss: 0.0930
[50000/60000 ( 83%)]  Loss: 0.0350

Average test loss: 0.1440  Accuracy: 9565/10000 (95.65%)

Epoch: 3
[    0/60000 (  0%)]  Loss: 0.1443
[10000/60000 ( 17%)]  Loss: 0.1801
[20000/60000 ( 33%)]  Loss: 0.1536
[30000/60000 ( 50%)]  Loss: 0.0864
[40000/60000 ( 67%)]  Loss: 0.1120
[50000/60000 ( 83%)]  Loss: 0.1194

Average test loss: 0.1251  Accuracy: 9620/10000 (96.20%)

Epoch: 4
[    0/60000 (  0%)]  Loss: 0.0724
[10000/60000 ( 17%)]  Loss: 0.1834
[20000/60000 ( 33%)]  Loss: 0.0861
[30000/60000 ( 50%)]  Loss: 0.0421
[40000/60000 ( 67

In [9]:
t1 = torch.ones(1,3,4)
t3 = torch.ones(1,3,4) * 2
t1 = torch.cat((t1,t3), dim=0)

In [10]:
t1.shape

torch.Size([2, 3, 4])

In [11]:
t1

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[2., 2., 2., 2.],
         [2., 2., 2., 2.],
         [2., 2., 2., 2.]]])

In [12]:
t2 = t1.view(t1.shape[0]* t1.shape[1], t1.shape[2])

In [13]:
t2

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]])

In [14]:
t2.shape

torch.Size([6, 4])

In [15]:
t4 = t2.view((2, 3, 4))
print(t4.shape)

torch.Size([2, 3, 4])


In [16]:
x = torch.zeros(2, t2.shape[0] * t1.shape[1])
for i in range(t2.shape[0]):
    for j in range(t1.shape[1]):
#         print(f'v1 {i}')
#         print(f'v2 {t1.shape[1] * (i // t1.shape[1]) + j}\n')
        x[0, i*t1.shape[1] + j] = i
        x[1, i*t1.shape[1] + j] = t1.shape[1] * (i // t1.shape[1]) + j
print(x)

tensor([[0., 0., 0., 1., 1., 1., 2., 2., 2., 3., 3., 3., 4., 4., 4., 5., 5., 5.],
        [0., 1., 2., 0., 1., 2., 0., 1., 2., 3., 4., 5., 3., 4., 5., 3., 4., 5.]])
